In [1]:
# %%
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pyplot as plt
#import seaborn as sns
import pickle



# %%
YEARS = [2018, 2019, 2020, 2021, 2022, 2023,2024]

# %%
data_all = pd.DataFrame()

def calculate_seconds(row):
    if row['qtr'] != 5:
        return 3600 - row['game_seconds_remaining']
    else:
        return 600 - row['game_seconds_remaining'] + 3600


def get_quarter_value(dataf):
    if 'END QUARTER' in dataf['desc']:
        return dataf['level_0']
    else:
        return None

for i in YEARS:  
    i_data = pd.read_csv('https://github.com/nflverse/nflverse-data/releases/download/pbp/' \
                   'play_by_play_' + str(i) + '.csv.gz',
                   compression= 'gzip', low_memory= False)

    data_all = pd.concat([data_all,i_data])

ppr = 1

data = data_all.loc[data_all.season_type=='REG']
#data = data_all.loc[(data_all.play_type.isin(['no_play','pass','run'])) & (data_all.epa.isna()==False)]
#data.loc[data['pass']==1, 'play_type'] = 'pass'
#data.loc[data.rush==1, 'play_type'] = 'run'
data.reset_index(drop=True, inplace=True)
data['turnover'] = data['interception'] + data['fumble_lost']
data = data.dropna(subset=['posteam'])
data['inside_10'] = (data['yardline_100'] < 10).astype(int)
data['20+_play'] = (data['yards_gained'] > 19).astype(int)
data['short_pass'] = (data['air_yards'] < 10).astype(int)
data['medium_pass'] = ((data['air_yards'] > 9)&(data['air_yards']<20)).astype(int)
data['deep_pass'] = (data['air_yards'] > 19).astype(int)
data['end_zone_target'] = (data['yardline_100'] - data['air_yards']) <= 0
data['fantasy_points'] = (
    data['complete_pass'] * ppr +          # 1 point per completion
    data['touchdown'] * 6 +           # 6 points per touchdown
    data['yards_gained'] * 0.1        # 0.1 points per yard gained
)
data['distance_to_EZ_after_target'] = data['yardline_100'] - data['air_yards']


C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2018476112.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2018476112.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['turnover'] = data['interception'] + data['fumble_lost']
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2018476112.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [2]:
def total_finder(home_or_away,home_total,away_total):
    if home_or_away == 'home':
        total = home_total
    else:
        total = away_total 
    return total

In [3]:
    data.reset_index(drop=True, inplace=True)

    data = data[data['two_point_attempt']==0]


    # derive implied team total from betting market data
    data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
    data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)

    # Use list comprehension with zip for more efficient row-wise operations
    data['implied_posteam_total'] = [
    total_finder(has_ball, home_number, away_number)
        for has_ball, home_number, away_number in zip(data['posteam_type'], data['home_implied_total'], data['away_implied_total'])
    ]

C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2215997381.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['home_implied_total'] = abs(data['total_line'] / 2 + data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2215997381.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['away_implied_total'] = abs(data['total_line'] / 2 - data['spread_line'] / 2)
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\2215997381.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the r

In [4]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


# Season-over-season Comparison

In [5]:
def load_models():
    with open('Models/yardage_model.pkl', 'rb') as file:
        yardage_model = pickle.load(file)
    
    with open('Models/touchdown_model.pkl', 'rb') as file:
        touchdown_model = pickle.load(file)

    return yardage_model, touchdown_model

In [6]:
yardage_model, touchdown_model = load_models()

In [7]:
df['season'].value_counts()

season
2021    18055
2023    17483
2020    17307
2022    17305
2018    17172
2019    17142
Name: count, dtype: int64

In [8]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,0.0,False,72.0
5,4.0,39.0,10,1.0,right,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,0.0,False,42.0
7,24.0,39.0,10,3.0,left,0.0,False,15.0
10,1.0,1.0,1,3.0,right,0.0,True,0.0


In [9]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


In [10]:
# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_18452\203326306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\203326306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:, 1]
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\203326306.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [11]:
receivers = df.groupby(['receiver_player_id','posteam','season']).agg({'receiver_player_name':'max','pass':'sum','xTDs':'sum','pass_touchdown':'sum','xYards':'sum','yards_gained':'sum','cp':'sum','complete_pass':'sum','fantasy_points':'sum','compositeXFP':'sum'}).sort_values('compositeXFP',ascending=False)

In [12]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.596006,16.0,1438.502686,1947.0,129.081419,145.0,441.7,342.507733
00-0036358,DAL,2023,C.Lamb,181,12.352115,12.0,1393.984985,1749.0,117.402901,135.0,381.9,330.914094
00-0031381,LV,2022,D.Adams,180,10.494546,14.0,1526.758789,1516.0,112.967685,100.0,341.6,328.610843
00-0036322,MIN,2022,J.Jefferson,184,10.750579,8.0,1458.707886,1819.0,116.943704,128.0,357.9,327.317971
00-0032765,NO,2019,M.Thomas,185,7.032757,9.0,1456.816895,1725.0,128.447433,149.0,375.5,316.325664
...,...,...,...,...,...,...,...,...,...,...,...,...
00-0035289,JAX,2020,G.Minshew II,1,0.000601,0.0,0.969162,0.0,0.596584,1.0,1.0,0.697108
00-0019596,NE,2018,T.Brady,1,0.002768,0.0,3.223743,6.0,0.329838,1.0,1.6,0.668823
00-0029263,SEA,2018,R.Wilson,1,0.002653,0.0,0.449581,-11.0,0.570121,1.0,-0.1,0.630997


In [13]:
receivers_qual = receivers[receivers['pass']>=20]

In [14]:
receivers

,,,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
receiver_player_id,posteam,season,,,,,,,,,,
00-0033908,LA,2021,C.Kupp,191,11.596006,16.0,1438.502686,1947.0,129.081419,145.0,441.7,342.507733
00-0036358,DAL,2023,C.Lamb,181,12.352115,12.0,1393.984985,1749.0,117.402901,135.0,381.9,330.914094
00-0031381,LV,2022,D.Adams,180,10.494546,14.0,1526.758789,1516.0,112.967685,100.0,341.6,328.610843
00-0036322,MIN,2022,J.Jefferson,184,10.750579,8.0,1458.707886,1819.0,116.943704,128.0,357.9,327.317971
00-0032765,NO,2019,M.Thomas,185,7.032757,9.0,1456.816895,1725.0,128.447433,149.0,375.5,316.325664
...,...,...,...,...,...,...,...,...,...,...,...,...
00-0035289,JAX,2020,G.Minshew II,1,0.000601,0.0,0.969162,0.0,0.596584,1.0,1.0,0.697108
00-0019596,NE,2018,T.Brady,1,0.002768,0.0,3.223743,6.0,0.329838,1.0,1.6,0.668823
00-0029263,SEA,2018,R.Wilson,1,0.002653,0.0,0.449581,-11.0,0.570121,1.0,-0.1,0.630997


In [15]:
receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,complete_pass,fantasy_points,compositeXFP
800,00-0021547,LAC,2018,A.Gates,45,3.752923,2.0,308.335419,333.0,27.173962,28.0,79.3,80.525043
402,00-0022127,DAL,2019,J.Witten,83,3.288441,4.0,569.323181,529.0,57.494382,63.0,139.9,134.157347
162,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,69.0,178.4,195.719583
166,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
489,00-0022921,ARI,2020,L.Fitzgerald,72,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,00-0039075,LA,2023,P.Nacua,160,6.651248,6.0,1286.854736,1486.0,108.953619,105.0,289.6,277.546585
581,00-0039139,DET,2023,J.Gibbs,71,1.722214,1.0,384.768494,316.0,56.332290,52.0,95.6,105.142424
864,00-0039144,GB,2023,L.Musgrave,46,1.390252,1.0,346.595734,352.0,31.674964,34.0,75.2,74.676047
244,00-0039146,GB,2023,J.Reed,94,5.999262,8.0,733.035645,793.0,59.541225,64.0,191.3,168.840368


In [16]:
df = receivers_qual.reset_index().sort_values(['receiver_player_id', 'season'])

# Create lagged columns for the next season's stats
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass','fantasy_points', 'compositeXFP']:
    df[f'{col}_next'] = df.groupby('receiver_player_id')[col].shift(-1)

# Drop rows where the next season's data is missing (last season for each player)
df_lagged = df.dropna(subset=[f'{col}_next' for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass','fantasy_points', 'compositeXFP']])

In [17]:
df.head(20)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
800,00-0021547,LAC,2018,A.Gates,45,3.752923,2.0,308.335419,333.0,27.173962,...,79.3,80.525043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,00-0022127,DAL,2019,J.Witten,83,3.288441,4.0,569.323181,529.0,57.494382,...,139.9,134.157347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
166,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
489,00-0022921,ARI,2020,L.Fitzgerald,72,2.098424,1.0,524.114929,409.0,52.608162,...,100.9,117.610205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
699,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1343,00-0022943,NE,2019,B.Watson,24,1.276046,0.0,177.596542,173.0,15.453680,...,34.3,40.869614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1412,00-0023564,PHI,2018,D.Sproles,23,1.313301,2.0,121.192467,160.0,17.396862,...,43.0,37.395913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,00-0024221,WAS,2018,V.Davis,36,0.967655,2.0,311.169250,367.0,22.395950,...,73.7,59.318803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1203,00-0024243,GB,2021,M.Lewis,28,2.228251,0.0,160.090912,214.0,19.881775,...,44.4,49.260372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
162,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
166,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
699,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1426,00-0025394,WAS,2018,A.Peterson,26,0.391267,1.0,141.563919,208.0,20.414077,...,46.8,36.918073,0.355608,0.0,106.821838,142.0,17.924322,17.0,31.2,30.740152
1068,00-0025396,NO,2018,T.Ginn,30,2.191082,2.0,278.764801,209.0,17.385297,...,49.9,58.408270,3.995163,2.0,496.046753,421.0,32.226284,30.0,84.1,105.801939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,00-0038117,NYG,2022,W.Robinson,31,0.639562,1.0,224.809448,227.0,22.078208,...,51.7,48.396525,1.039892,1.0,530.590881,525.0,56.055348,60.0,118.5,115.353791
1220,00-0038120,NYJ,2022,Br.Hall,31,1.210813,1.0,191.749786,218.0,21.799425,...,46.8,48.239283,1.083842,4.0,499.608429,591.0,73.959531,76.0,159.1,130.423429
428,00-0038124,GB,2022,C.Watson,66,5.672565,7.0,567.389343,611.0,38.034695,...,144.1,128.809020,6.244337,5.0,442.108673,422.0,28.629502,28.0,100.2,110.306393
514,00-0038129,TB,2022,C.Otton,65,4.337287,2.0,438.540283,391.0,45.103641,...,93.1,114.981395,3.481099,4.0,456.120300,452.0,46.431071,47.0,116.2,112.929696


In [19]:
df_lagged.tail(11)

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
519,00-0037816,GB,2022,R.Doubs,67,2.378791,3.0,541.901794,425.0,44.964937,...,102.5,113.427863,7.307990,8.0,778.729858,674.0,57.829159,59.0,174.4,179.550084
597,00-0037838,BAL,2022,I.Likely,60,3.336625,3.0,431.971924,373.0,40.330922,...,91.3,103.547865,1.449008,5.0,285.263336,411.0,27.930377,30.0,101.1,65.150761
1442,00-0038041,DAL,2022,J.Ferguson,22,0.842659,2.0,140.499924,174.0,16.795957,...,48.4,35.901905,6.951853,5.0,646.522583,761.0,71.300789,71.0,177.1,177.664168
1136,00-0038090,KC,2022,S.Moore,33,0.614525,0.0,266.729248,250.0,23.407232,...,47.0,53.767305,1.936463,1.0,289.558472,244.0,24.452009,21.0,51.4,65.026631
737,00-0038104,NE,2022,T.Thornton,45,3.312339,2.0,405.881989,247.0,26.600075,...,58.7,87.062307,0.558417,0.0,184.082565,91.0,15.103351,13.0,22.1,36.862109
1162,00-0038115,NYG,2022,D.Bellinger,35,0.818270,2.0,222.478012,268.0,24.952157,...,68.8,52.109576,0.132520,0.0,195.951233,255.0,21.681777,25.0,50.5,42.072021
1217,00-0038117,NYG,2022,W.Robinson,31,0.639562,1.0,224.809448,227.0,22.078208,...,51.7,48.396525,1.039892,1.0,530.590881,525.0,56.055348,60.0,118.5,115.353791
1220,00-0038120,NYJ,2022,Br.Hall,31,1.210813,1.0,191.749786,218.0,21.799425,...,46.8,48.239283,1.083842,4.0,499.608429,591.0,73.959531,76.0,159.1,130.423429
428,00-0038124,GB,2022,C.Watson,66,5.672565,7.0,567.389343,611.0,38.034695,...,144.1,128.809020,6.244337,5.0,442.108673,422.0,28.629502,28.0,100.2,110.306393
514,00-0038129,TB,2022,C.Otton,65,4.337287,2.0,438.540283,391.0,45.103641,...,93.1,114.981395,3.481099,4.0,456.120300,452.0,46.431071,47.0,116.2,112.929696


In [20]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for col in ['xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass', 'fantasy_points', 'compositeXFP']:
    correlation = df_lagged[col].corr(df_lagged[f'{col}_next'])
    #print(df_lagged)
    correlations[col] = correlation

# Display the results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation with Next Season'])
print(correlations_df)

                Correlation with Next Season
xTDs                                0.642631
pass_touchdown                      0.487490
xYards                              0.714351
yards_gained                        0.687922
cp                                  0.611071
complete_pass                       0.620002
fantasy_points                      0.662808
compositeXFP                        0.679733


In [21]:
#correlations_df.drop(['xFPs'],inplace=True)

In [22]:
correlations = {}

# Loop through each stat to calculate its correlation with the following season
for metric in ['pass_touchdown', 'yards_gained', 'complete_pass','fantasy_points']:
    if metric == 'pass_touchdown':
        predictor = 'xTDs'
    if metric == 'yards_gained':
        predictor = 'xYards'
    if metric == 'complete_pass':
        predictor = 'cp'
    if metric == 'fantasy_points':
        predictor = 'compositeXFP'
    #print(f"{predictor} to predict {metric}")
    correlation = df_lagged[predictor].corr(df_lagged[f'{metric}_next'])
    correlations[metric] = correlation

# Display the results
x_stat_correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Expected Stat Correlation with Actual Next Season Target'])
print(x_stat_correlations_df)

                Expected Stat Correlation with Actual Next Season Target
pass_touchdown                                           0.517943       
yards_gained                                             0.679986       
complete_pass                                            0.604706       
fantasy_points                                           0.648267       


In [23]:
correlations_df.merge(x_stat_correlations_df,right_index=True,left_index=True,how='outer').sort_values('Correlation with Next Season',ascending=False)

,Correlation with Next Season,Expected Stat Correlation with Actual Next Season Target
xYards,0.714351,NaN
yards_gained,0.687922,0.679986
compositeXFP,0.679733,NaN
fantasy_points,0.662808,0.648267
xTDs,0.642631,NaN
complete_pass,0.620002,0.604706
cp,0.611071,NaN
pass_touchdown,0.487490,0.517943


In [24]:
df_lagged[['xTDs','pass_touchdown','pass_touchdown_next']].corr()

,xTDs,pass_touchdown,pass_touchdown_next
xTDs,1.000000,0.82115,0.517943
pass_touchdown,0.821150,1.00000,0.487490
pass_touchdown_next,0.517943,0.48749,1.000000


In [25]:
df_lagged[['xYards','yards_gained','yards_gained_next']].corr()

,xYards,yards_gained,yards_gained_next
xYards,1.000000,0.962452,0.679986
yards_gained,0.962452,1.000000,0.687922
yards_gained_next,0.679986,0.687922,1.000000


In [26]:
df_lagged

,receiver_player_id,posteam,season,receiver_player_name,pass,xTDs,pass_touchdown,xYards,yards_gained,cp,...,fantasy_points,compositeXFP,xTDs_next,pass_touchdown_next,xYards_next,yards_gained_next,cp_next,complete_pass_next,fantasy_points_next,compositeXFP_next
162,00-0022921,ARI,2018,L.Fitzgerald,112,5.852049,6.0,880.626404,734.0,72.544649,...,178.4,195.719583,6.226891,4.0,826.374084,804.0,74.956883,75.0,191.4,194.955640
166,00-0022921,ARI,2019,L.Fitzgerald,109,6.226891,4.0,826.374084,804.0,74.956883,...,191.4,194.955640,2.098424,1.0,524.114929,409.0,52.608162,54.0,100.9,117.610205
699,00-0022943,NO,2018,B.Watson,46,4.236354,2.0,367.110840,400.0,29.137302,...,87.0,91.266512,1.276046,0.0,177.596542,173.0,15.453680,17.0,34.3,40.869614
1426,00-0025394,WAS,2018,A.Peterson,26,0.391267,1.0,141.563919,208.0,20.414077,...,46.8,36.918073,0.355608,0.0,106.821838,142.0,17.924322,17.0,31.2,30.740152
1068,00-0025396,NO,2018,T.Ginn,30,2.191082,2.0,278.764801,209.0,17.385297,...,49.9,58.408270,3.995163,2.0,496.046753,421.0,32.226284,30.0,84.1,105.801939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,00-0038117,NYG,2022,W.Robinson,31,0.639562,1.0,224.809448,227.0,22.078208,...,51.7,48.396525,1.039892,1.0,530.590881,525.0,56.055348,60.0,118.5,115.353791
1220,00-0038120,NYJ,2022,Br.Hall,31,1.210813,1.0,191.749786,218.0,21.799425,...,46.8,48.239283,1.083842,4.0,499.608429,591.0,73.959531,76.0,159.1,130.423429
428,00-0038124,GB,2022,C.Watson,66,5.672565,7.0,567.389343,611.0,38.034695,...,144.1,128.809020,6.244337,5.0,442.108673,422.0,28.629502,28.0,100.2,110.306393
514,00-0038129,TB,2022,C.Otton,65,4.337287,2.0,438.540283,391.0,45.103641,...,93.1,114.981395,3.481099,4.0,456.120300,452.0,46.431071,47.0,116.2,112.929696


# Intra-season Comparison

In [27]:
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    #throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [28]:
new_predictors = [
    'air_yards', 'yardline_100', 'ydstogo',
    'down', 'pass_location', 'qb_hit', 'end_zone_target', 'distance_to_EZ_after_target'
]

test_df = df[new_predictors]

test_df.head()

,air_yards,yardline_100,ydstogo,down,pass_location,qb_hit,end_zone_target,distance_to_EZ_after_target
2,8.0,80.0,15,1.0,right,0.0,False,72.0
5,4.0,39.0,10,1.0,right,0.0,False,35.0
6,-3.0,39.0,10,2.0,left,0.0,False,42.0
7,24.0,39.0,10,3.0,left,0.0,False,15.0
10,1.0,1.0,1,3.0,right,0.0,True,0.0


In [29]:
    
    # we only want throws to a receiver, aka plays with air yardage (no running plays, sacks, throwaways etc.)
    throws = data[data['air_yards'].notna()]
    # only data before the current szn
    #throws = throws[throws['season']!=2024]
    throws = throws[throws['receiver_player_name'].notna()]
    throws = throws[throws['pass_location'].notna()]

    
    df = throws[['receiver_player_name','receiver_player_id','posteam','pass','cp','game_id','complete_pass','inside_10','air_yards','yardline_100','ydstogo','implied_posteam_total','yards_gained','fantasy_points','pass_touchdown','down','pass_location','week','season','home_implied_total','away_implied_total','posteam_type','qb_hit','end_zone_target', 'distance_to_EZ_after_target']]


In [30]:
test_df = pd.get_dummies(test_df, columns=['pass_location'], drop_first=True)


# Add predictions to the new dataset (optional)
df['xYards'] = yardage_model.predict(test_df)
df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
#df['xFPs'] = best_model.predict(test_df)
df['compositeXFP'] = df['cp'] * ppr + df['xTDs'] * 6 + df['xYards'] * 0.1

C:\Users\src15\AppData\Local\Temp\ipykernel_18452\896432768.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xYards'] = yardage_model.predict(test_df)
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\896432768.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['xTDs'] = touchdown_model.predict_proba(test_df)[:,1]
C:\Users\src15\AppData\Local\Temp\ipykernel_18452\896432768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [31]:

# Filter weeks 1-9 and weeks 10-18
first_half = df[df['week'].between(1, 9)]
second_half = df[df['week'].between(10, 18)]

# Define metrics to aggregate
metrics = ['pass','xTDs', 'pass_touchdown', 'xYards', 'yards_gained', 'cp', 'complete_pass',  'fantasy_points', 'compositeXFP','end_zone_target']

# Aggregate metrics for each player-season for weeks 1-9
first_half_agg = first_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
first_half_agg = first_half_agg.add_suffix('_wk1_9')

# Aggregate metrics for each player-season for weeks 10-18
second_half_agg = second_half.groupby(['receiver_player_id', 'season'])[metrics].sum()
second_half_agg = second_half_agg.add_suffix('_wk10_18')

# Merge the two aggregated DataFrames
df_aggregated = pd.merge(first_half_agg, second_half_agg, left_index=True, right_index=True)


In [32]:
df_aggregated = df_aggregated[(df_aggregated['pass_wk1_9'] >= 5) & (df_aggregated['pass_wk10_18'] >= 5)]


In [33]:
# Create a dictionary to store correlations for each metric
correlations = {}

# Calculate correlation for each metric between weeks 1-9 and weeks 10-18
for metric in metrics:
    correlation = df_aggregated[f'{metric}_wk1_9'].corr(df_aggregated[f'{metric}_wk10_18'])
    correlations[metric] = correlation

# Display the correlation results
correlations_df = pd.DataFrame.from_dict(correlations, orient='index', columns=['Correlation between Weeks 1-9 and 10-18'])
print(correlations_df)


                 Correlation between Weeks 1-9 and 10-18
pass                                            0.711110
xTDs                                            0.570537
pass_touchdown                                  0.427380
xYards                                          0.750812
yards_gained                                    0.706406
cp                                              0.686568
complete_pass                                   0.683898
fantasy_points                                  0.685264
compositeXFP                                    0.719506
end_zone_target                                 0.502806


In [34]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','end_zone_target_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,pass_wk1_9,end_zone_target_wk1_9
fantasy_points_wk1_9,1.000000,0.954680,0.685264,0.947273,0.675276
compositeXFP_wk1_9,0.954680,1.000000,0.681885,0.990141,0.718748
fantasy_points_wk10_18,0.685264,0.681885,1.000000,0.672112,0.484984
pass_wk1_9,0.947273,0.990141,0.672112,1.000000,0.641261
end_zone_target_wk1_9,0.675276,0.718748,0.484984,0.641261,1.000000


In [35]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18','end_zone_target_wk1_9']].corr()[['fantasy_points_wk10_18']].sort_values('fantasy_points_wk10_18',ascending=False)

,fantasy_points_wk10_18
fantasy_points_wk10_18,1.000000
yards_gained_wk10_18,0.974119
pass_wk10_18,0.948361
pass_touchdown_wk10_18,0.822624
yards_gained_wk1_9,0.689174
fantasy_points_wk1_9,0.685264
compositeXFP_wk1_9,0.681885
xYards_wk1_9,0.681601
pass_wk1_9,0.672112
xTDs_wk1_9,0.586152


In [36]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18']].corr()[['yards_gained_wk10_18']].sort_values('yards_gained_wk10_18',ascending=False)

,yards_gained_wk10_18
yards_gained_wk10_18,1.000000
fantasy_points_wk10_18,0.974119
pass_wk10_18,0.932635
pass_touchdown_wk10_18,0.722741
yards_gained_wk1_9,0.706406
xYards_wk1_9,0.701529
compositeXFP_wk1_9,0.692194
fantasy_points_wk1_9,0.690736
pass_wk1_9,0.674954
xTDs_wk1_9,0.605445


In [37]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9','pass_wk10_18','yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18','xTDs_wk1_9','pass_touchdown_wk1_9','pass_touchdown_wk10_18','end_zone_target_wk1_9']].corr()[['pass_touchdown_wk10_18']].sort_values('pass_touchdown_wk10_18',ascending=False)

,pass_touchdown_wk10_18
pass_touchdown_wk10_18,1.000000
fantasy_points_wk10_18,0.822624
yards_gained_wk10_18,0.722741
pass_wk10_18,0.670831
fantasy_points_wk1_9,0.491995
yards_gained_wk1_9,0.491423
xYards_wk1_9,0.483470
compositeXFP_wk1_9,0.479218
pass_wk1_9,0.458281
xTDs_wk1_9,0.455795


In [38]:
df_aggregated[['yards_gained_wk1_9','xYards_wk1_9','yards_gained_wk10_18']].corr()

,yards_gained_wk1_9,xYards_wk1_9,yards_gained_wk10_18
yards_gained_wk1_9,1.000000,0.948712,0.706406
xYards_wk1_9,0.948712,1.000000,0.701529
yards_gained_wk10_18,0.706406,0.701529,1.000000


In [39]:
df_aggregated[['pass_touchdown_wk1_9','xTDs_wk1_9','pass_touchdown_wk10_18']].corr()

,pass_touchdown_wk1_9,xTDs_wk1_9,pass_touchdown_wk10_18
pass_touchdown_wk1_9,1.000000,0.779691,0.427380
xTDs_wk1_9,0.779691,1.000000,0.455795
pass_touchdown_wk10_18,0.427380,0.455795,1.000000


In [40]:
df_aggregated[['fantasy_points_wk1_9','compositeXFP_wk1_9','fantasy_points_wk10_18','pass_wk1_9']].corr()

,fantasy_points_wk1_9,compositeXFP_wk1_9,fantasy_points_wk10_18,pass_wk1_9
fantasy_points_wk1_9,1.000000,0.954680,0.685264,0.947273
compositeXFP_wk1_9,0.954680,1.000000,0.681885,0.990141
fantasy_points_wk10_18,0.685264,0.681885,1.000000,0.672112
pass_wk1_9,0.947273,0.990141,0.672112,1.000000
